In [1]:
!pip install pyspark
!pip install py4j
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www.apache.org/dyn/closer.lua/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 61.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a32435c549bed369cb668ca0a4df1b4211b9bb89c85270ac1800edc120536831
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
movies = files.upload()

Saving movies.csv to movies.csv


In [42]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local") \
.appName("trial")  \
.config('spark.ui.port','4050') \
.getOrCreate()

In [43]:
spark

In [44]:
dataframe = spark.read.csv('movies.csv')
dataframe.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [45]:
dataframe = spark.read.format("csv").option("header","false").load("movies.csv")
dataframe.show()

+--------------------+--------------------+----+
|                 _c0|                 _c1| _c2|
+--------------------+--------------------+----+
|1::Toy Story (199...|                null|null|
|2::Jumanji (1995)...|                null|null|
|3::Grumpier Old M...|                null|null|
|4::Waiting to Exh...|                null|null|
|5::Father of the ...|                null|null|
|6::Heat (1995)::A...|                null|null|
|7::Sabrina (1995)...|                null|null|
|8::Tom and Huck (...|                null|null|
|9::Sudden Death (...|                null|null|
|10::GoldenEye (19...|                null|null|
|11::American Pres...| The (1995)::Come...|null|
|12::Dracula: Dead...|                null|null|
|13::Balto (1995):...|                null|null|
|14::Nixon (1995):...|                null|null|
|15::Cutthroat Isl...|                null|null|
|16::Casino (1995)...|                null|null|
|17::Sense and Sen...|                null|null|
|18::Four Rooms (1..

In [46]:
from pyspark.context import SparkContext
import pyspark.sql
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [47]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col 
import pyspark.sql.functions as f

In [48]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [49]:
schema = StructType([
    StructField("Id",IntegerType(),True), \
    StructField("movie_name",StringType(),True), \
    StructField("movies genre",StringType(),True),\
    StructField("year2",IntegerType(),True),
                     ])

In [50]:
df=spark.read.csv("movies.csv", sep="::",schema=schema)
df.show(truncate = False)

+---+-------------------------------------+------------------------------+-----+
|Id |movie_name                           |movies genre                  |year2|
+---+-------------------------------------+------------------------------+-----+
|1  |Toy Story (1995)                     |Animation|Children's|Comedy,, |null |
|2  |Jumanji (1995)                       |Adventure|Children's|Fantasy,,|null |
|3  |Grumpier Old Men (1995)              |Comedy|Romance,,              |null |
|4  |Waiting to Exhale (1995)             |Comedy|Drama,,                |null |
|5  |Father of the Bride Part II (1995)   |Comedy,,                      |null |
|6  |Heat (1995)                          |Action|Crime|Thriller,,       |null |
|7  |Sabrina (1995)                       |Comedy|Romance,,              |null |
|8  |Tom and Huck (1995)                  |Adventure|Children's,,        |null |
|9  |Sudden Death (1995)                  |Action,,                      |null |
|10 |GoldenEye (1995)       

In [51]:
df3 = df.withColumn('year2', split(df['movie_name'], " ").getItem(2)).show()


+---+--------------------+--------------------+-----------+
| Id|          movie_name|        movies genre|      year2|
+---+--------------------+--------------------+-----------+
|  1|    Toy Story (1995)|Animation|Childre...|     (1995)|
|  2|      Jumanji (1995)|Adventure|Childre...|       null|
|  3|Grumpier Old Men ...|    Comedy|Romance,,|        Men|
|  4|Waiting to Exhale...|      Comedy|Drama,,|     Exhale|
|  5|Father of the Bri...|            Comedy,,|        the|
|  6|         Heat (1995)|Action|Crime|Thri...|       null|
|  7|      Sabrina (1995)|    Comedy|Romance,,|       null|
|  8| Tom and Huck (1995)|Adventure|Childre...|       Huck|
|  9| Sudden Death (1995)|            Action,,|     (1995)|
| 10|    GoldenEye (1995)|Action|Adventure|...|       null|
| 11|American Presiden...|Comedy|Drama|Roma...|        The|
| 12|Dracula: Dead and...|     Comedy|Horror,,|        and|
| 13|        Balto (1995)|Animation|Childre...|       null|
| 14|        Nixon (1995)|             D

In [52]:

df2 = df.select(split(col("movie_name")," ").getItem(2).alias("year")).show()

+-----------+
|       year|
+-----------+
|     (1995)|
|       null|
|        Men|
|     Exhale|
|        the|
|       null|
|       null|
|       Huck|
|     (1995)|
|       null|
|        The|
|        and|
|       null|
|       null|
|     (1995)|
|       null|
|Sensibility|
|     (1995)|
|       When|
|     (1995)|
+-----------+
only showing top 20 rows



In [53]:
df3 = df.withColumn('year2', split(df['movie_name'], " ").getItem(1)).show(1)

"""
.withColumn('year3', split(df['movie_name'], " ").getItem(1)) \
.withColumn('year4', split(df['movie_name'], " ").getItem(2)) \
.withColumn('year5', split(df['movie_name'], " ").getItem(3)) \
.withColumn('year6', split(df['movie_name'], " ").getItem(4)) \
.withColumn('year7', split(df['movie_name'], " ").getItem(5)).show()
"""



+---+----------------+--------------------+-----+
| Id|      movie_name|        movies genre|year2|
+---+----------------+--------------------+-----+
|  1|Toy Story (1995)|Animation|Childre...|Story|
+---+----------------+--------------------+-----+
only showing top 1 row



'\n.withColumn(\'year3\', split(df[\'movie_name\'], " ").getItem(1)) .withColumn(\'year4\', split(df[\'movie_name\'], " ").getItem(2)) .withColumn(\'year5\', split(df[\'movie_name\'], " ").getItem(3)) .withColumn(\'year6\', split(df[\'movie_name\'], " ").getItem(4)) .withColumn(\'year7\', split(df[\'movie_name\'], " ").getItem(5)).show()\n'

In [81]:
df4 = df.withColumn("Year_Release", f.regexp_extract(df['movie_name'], '[1-9][0-9]*',0))
df4.show(5)

+---+--------------------+--------------------+-----+------------+
| Id|          movie_name|        movies genre|year2|Year_Release|
+---+--------------------+--------------------+-----+------------+
|  1|    Toy Story (1995)|Animation|Childre...| null|        1995|
|  2|      Jumanji (1995)|Adventure|Childre...| null|        1995|
|  3|Grumpier Old Men ...|    Comedy|Romance,,| null|        1995|
|  4|Waiting to Exhale...|      Comedy|Drama,,| null|        1995|
|  5|Father of the Bri...|            Comedy,,| null|        1995|
+---+--------------------+--------------------+-----+------------+
only showing top 5 rows



In [82]:
df4=df4.drop('year2')


In [83]:
df4.show()

+---+--------------------+--------------------+------------+
| Id|          movie_name|        movies genre|Year_Release|
+---+--------------------+--------------------+------------+
|  1|    Toy Story (1995)|Animation|Childre...|        1995|
|  2|      Jumanji (1995)|Adventure|Childre...|        1995|
|  3|Grumpier Old Men ...|    Comedy|Romance,,|        1995|
|  4|Waiting to Exhale...|      Comedy|Drama,,|        1995|
|  5|Father of the Bri...|            Comedy,,|        1995|
|  6|         Heat (1995)|Action|Crime|Thri...|        1995|
|  7|      Sabrina (1995)|    Comedy|Romance,,|        1995|
|  8| Tom and Huck (1995)|Adventure|Childre...|        1995|
|  9| Sudden Death (1995)|            Action,,|        1995|
| 10|    GoldenEye (1995)|Action|Adventure|...|        1995|
| 11|American Presiden...|Comedy|Drama|Roma...|        1995|
| 12|Dracula: Dead and...|     Comedy|Horror,,|        1995|
| 13|        Balto (1995)|Animation|Childre...|        1995|
| 14|        Nixon (1995

In [84]:
df5 = df4.withColumn("Name", f.regexp_extract(df['movie_name'], '[A-Z][a-z]*',0)).show()

+---+--------------------+--------------------+------------+---------+
| Id|          movie_name|        movies genre|Year_Release|     Name|
+---+--------------------+--------------------+------------+---------+
|  1|    Toy Story (1995)|Animation|Childre...|        1995|      Toy|
|  2|      Jumanji (1995)|Adventure|Childre...|        1995|  Jumanji|
|  3|Grumpier Old Men ...|    Comedy|Romance,,|        1995| Grumpier|
|  4|Waiting to Exhale...|      Comedy|Drama,,|        1995|  Waiting|
|  5|Father of the Bri...|            Comedy,,|        1995|   Father|
|  6|         Heat (1995)|Action|Crime|Thri...|        1995|     Heat|
|  7|      Sabrina (1995)|    Comedy|Romance,,|        1995|  Sabrina|
|  8| Tom and Huck (1995)|Adventure|Childre...|        1995|      Tom|
|  9| Sudden Death (1995)|            Action,,|        1995|   Sudden|
| 10|    GoldenEye (1995)|Action|Adventure|...|        1995|   Golden|
| 11|American Presiden...|Comedy|Drama|Roma...|        1995| American|
| 12|D

In [91]:
from pyspark.sql.functions import kurtosis	
from pyspark.sql.functions import min, max, mean
df4.select(max("Year_Release")).show()
df4.select(min("Year_Release")).show()


+-----------------+
|max(Year_Release)|
+-----------------+
|               98|
+-----------------+

+-----------------+
|min(Year_Release)|
+-----------------+
|                1|
+-----------------+



In [92]:
from google.colab import files
movies = files.upload()

Saving users.csv to users.csv
